In [1]:
!pip install pycaret

     |████████████████████████████████| 256kB 4.5MB/s 
     |████████████████████████████████| 1.6MB 15.6MB/s 
     |████████████████████████████████| 6.8MB 28.9MB/s 
     |████████████████████████████████| 13.9MB 303kB/s 
     |████████████████████████████████| 66.1MB 54kB/s 
     |████████████████████████████████| 102kB 11.6MB/s 
     |████████████████████████████████| 266kB 43.8MB/s 
     |████████████████████████████████| 266kB 47.9MB/s 
     |████████████████████████████████| 1.8MB 43.5MB/s 
     |████████████████████████████████| 174kB 48.5MB/s 
     |████████████████████████████████| 552kB 44.5MB/s 
     |████████████████████████████████| 81kB 9.4MB/s 
     |████████████████████████████████| 6.0MB 45.1MB/s 
     |████████████████████████████████| 153kB 46.0MB/s 
     |████████████████████████████████| 163kB 44.6MB/s 
     |████████████████████████████████| 327kB 46.2MB/s 
     |████████████████████████████████| 1.1MB 44.3MB/s 
     |████████████████████████████████| 51kB 6.2MB/s

In [2]:
from pycaret.datasets import get_data
data= get_data('employee')

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,department,salary,left
0,0.38,0.53,2,157,3,0,0,sales,low,1
1,0.80,0.86,5,262,6,0,0,sales,medium,1
2,0.11,0.88,7,272,4,0,0,sales,medium,1
3,0.72,0.87,5,223,5,0,0,sales,low,1
4,0.37,0.52,2,159,3,0,0,sales,low,1


In [3]:
data.columns

Index(['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident',
       'promotion_last_5years', 'department', 'salary', 'left'],
      dtype='object')

In [10]:
data.shape

(14999, 10)

In [4]:
data = data.rename(columns={'satisfaction_level':'niveau_satisfaction', 'last_evaluation':'derniere_evaluation', 'number_project':'nb_projet',
       'average_montly_hours':'heures_par_mois', 'time_spend_company':'anciennete', 'Work_accident':'accident_travail',
       'promotion_last_5years':'promotion_5ans', 'department':'departement', 'salary':'salaire', 'left':'quitte'})
data.head()

,niveau_satisfaction,derniere_evaluation,nb_projet,heures_par_mois,anciennete,accident_travail,promotion_5ans,departement,salaire,quitte
0,0.38,0.53,2,157,3,0,0,sales,low,1
1,0.80,0.86,5,262,6,0,0,sales,medium,1
2,0.11,0.88,7,272,4,0,0,sales,medium,1
3,0.72,0.87,5,223,5,0,0,sales,low,1
4,0.37,0.52,2,159,3,0,0,sales,low,1


In [5]:
data['departement'].unique()

array(['sales', 'accounting', 'hr', 'technical', 'support', 'management',
       'IT', 'product_mng', 'marketing', 'RandD'], dtype=object)

In [6]:
#dico pour remplacer les valeurs de departement
dep= {'sales': 'ventes', 'accounting':'comptabilite', 'hr':'rh', 'technical':'technique', 'support':'support', 'management':'management',
       'IT':'IT', 'product_mng':'product_mng', 'marketing':'marketing', 'RandD':'R&D'}

data['departement'].replace(dep, inplace=True)

In [7]:
#dico pour remplacer les valeurs de salaire
niveau= {'low':'bas', 'medium':'normal', 'high':'eleve'}
data['salaire'].replace(niveau, inplace=True)

In [8]:
data.describe()

,niveau_satisfaction,derniere_evaluation,nb_projet,heures_par_mois,anciennete,accident_travail,promotion_5ans,quitte
count,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000
mean,0.612834,0.716102,3.803054,201.050337,3.498233,0.144610,0.021268,0.238083
std,0.248631,0.171169,1.232592,49.943099,1.460136,0.351719,0.144281,0.425924
min,0.090000,0.360000,2.000000,96.000000,2.000000,0.000000,0.000000,0.000000
25%,0.440000,0.560000,3.000000,156.000000,3.000000,0.000000,0.000000,0.000000
50%,0.640000,0.720000,4.000000,200.000000,3.000000,0.000000,0.000000,0.000000
75%,0.820000,0.870000,5.000000,245.000000,4.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,7.000000,310.000000,10.000000,1.000000,1.000000,1.000000


In [9]:
data['quitte'].value_counts()

0    11428
1     3571
Name: quitte, dtype: int64

In [12]:
#Nous allons melanger le dataset afin de ne pas se retrouver avec un jeu de test avec une seule valeur
from sklearn.utils import shuffle
data = shuffle(data)
data.reset_index(inplace=True, drop=True) 
data

,niveau_satisfaction,derniere_evaluation,nb_projet,heures_par_mois,anciennete,accident_travail,promotion_5ans,departement,salaire,quitte
0,0.39,0.57,2,145,3,0,0,comptabilite,normal,1
1,0.32,0.41,3,138,3,1,0,technique,normal,0
2,0.98,0.98,4,170,10,0,0,IT,normal,0
3,0.83,0.48,4,220,3,1,0,support,bas,0
4,0.44,0.65,2,151,3,0,0,R&D,normal,0
...,...,...,...,...,...,...,...,...,...,...
14994,0.15,0.53,4,205,5,1,0,ventes,bas,0
14995,0.10,0.94,7,281,4,0,0,technique,normal,1
14996,0.90,0.48,4,136,7,0,0,IT,bas,0
14997,0.72,0.78,3,229,2,0,0,ventes,normal,0


In [14]:
categories= [ 
       'promotion_5ans', 'departement', 'salaire', 'quitte']
for cat in categories:
  print (data[cat].value_counts())

0    14680
1      319
Name: promotion_5ans, dtype: int64
ventes          4140
technique       2720
support         2229
IT              1227
product_mng      902
marketing        858
R&D              787
comptabilite     767
rh               739
management       630
Name: departement, dtype: int64
bas       7316
normal    6446
eleve     1237
Name: salaire, dtype: int64
0    11428
1     3571
Name: quitte, dtype: int64


In [15]:
df_model= data.sample(frac=0.95, random_state=16).reset_index(drop=True)
df_cache=data.drop(df_model.index).reset_index(drop=True)
print('Data pour Modeling : ' + str(df_model.shape))
print('Data pour Predictions : ' + str(df_cache.shape))

Data pour Modeling : (14249, 10)
Data pour Predictions : (750, 10)


In [16]:
df_cache['quitte'].value_counts()

0    550
1    200
Name: quitte, dtype: int64

In [17]:
df_cache.to_csv('df_cache.csv', index=False)
data.to_csv('churn_fr.csv', index=False)

In [18]:
from pycaret.classification import *
df= setup(data= df_model, target='quitte', session_id=16, numeric_features=['nb_projet', 'anciennete'])

Setup Succesfully Completed!


,Description,Value
0,session_id,16
1,Target Type,Binary
2,Label Encoded,"0: 0, 1: 1"
3,Original Data,"(14249, 10)"
4,Missing Values,False
5,Numeric Features,5
6,Categorical Features,4
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [19]:
compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,Light Gradient Boosting Machine,0.9848,0.9923,0.9508,0.9849,0.9675,0.9575,0.9578,0.1532
1,Random Forest Classifier,0.9844,0.9869,0.9436,0.9903,0.9664,0.9562,0.9567,0.1126
2,Extra Trees Classifier,0.9833,0.9897,0.9483,0.9809,0.9643,0.9533,0.9536,0.5727
3,CatBoost Classifier,0.9827,0.9908,0.9428,0.9838,0.9628,0.9515,0.9519,4.8592
4,Gradient Boosting Classifier,0.9758,0.9866,0.9298,0.9678,0.9483,0.9326,0.9329,1.0420
5,Extreme Gradient Boosting,0.9748,0.9858,0.9264,0.9668,0.9461,0.9297,0.9301,0.5187
6,Decision Tree Classifier,0.9739,0.9687,0.9588,0.9340,0.9461,0.9289,0.9292,0.0392
7,Ada Boost Classifier,0.9567,0.9803,0.8969,0.9197,0.9080,0.8797,0.8800,0.3982
8,K Neighbors Classifier,0.9301,0.9655,0.9146,0.8156,0.8620,0.8154,0.8181,0.0293
9,Logistic Regression,0.7954,0.8168,0.3738,0.6185,0.4644,0.3477,0.3652,0.2147


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=16, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [20]:
rf= create_model('rf')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9880,0.9886,0.9580,0.9913,0.9744,0.9665,0.9667
1,0.9910,0.9924,0.9622,1.0000,0.9807,0.9748,0.9752
2,0.9840,0.9902,0.9496,0.9826,0.9658,0.9553,0.9556
3,0.9840,0.9888,0.9328,1.0000,0.9652,0.9548,0.9558
4,0.9769,0.9834,0.9156,0.9864,0.9497,0.9347,0.9358
5,0.9870,0.9935,0.9536,0.9912,0.9720,0.9635,0.9639
6,0.9870,0.9884,0.9580,0.9870,0.9723,0.9638,0.9639
7,0.9809,0.9818,0.9244,0.9955,0.9586,0.9462,0.9473
8,0.9829,0.9800,0.9412,0.9868,0.9634,0.9523,0.9528
9,0.9819,0.9819,0.9412,0.9825,0.9614,0.9496,0.9500


In [21]:
rf_tuned= tune_model(rf)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9870,0.9898,0.9580,0.9870,0.9723,0.9638,0.9640
1,0.9920,0.9956,0.9706,0.9957,0.9830,0.9777,0.9779
2,0.9900,0.9925,0.9664,0.9914,0.9787,0.9722,0.9723
3,0.9860,0.9920,0.9412,1.0000,0.9697,0.9606,0.9613
4,0.9860,0.9866,0.9578,0.9827,0.9701,0.9609,0.9610
5,0.9920,0.9921,0.9662,1.0000,0.9828,0.9776,0.9778
6,0.9880,0.9938,0.9580,0.9913,0.9744,0.9665,0.9667
7,0.9779,0.9877,0.9160,0.9909,0.9520,0.9377,0.9389
8,0.9840,0.9903,0.9412,0.9912,0.9655,0.9551,0.9556
9,0.9799,0.9823,0.9370,0.9781,0.9571,0.9440,0.9444


In [22]:
lgb= create_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9850,0.9909,0.9580,0.9785,0.9682,0.9583,0.9584
1,0.9910,0.9943,0.9706,0.9914,0.9809,0.9750,0.9751
2,0.9840,0.9930,0.9622,0.9703,0.9662,0.9557,0.9557
3,0.9870,0.9954,0.9454,1.0000,0.9719,0.9635,0.9641
4,0.9819,0.9934,0.9451,0.9782,0.9614,0.9496,0.9498
5,0.9910,0.9949,0.9662,0.9957,0.9807,0.9748,0.9750
6,0.9870,0.9937,0.9538,0.9913,0.9722,0.9637,0.9640
7,0.9799,0.9891,0.9202,0.9955,0.9563,0.9433,0.9446
8,0.9819,0.9930,0.9412,0.9825,0.9614,0.9496,0.9500
9,0.9789,0.9857,0.9454,0.9657,0.9554,0.9416,0.9417


In [23]:
lgb_tuned= tune_model(lgb)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9870,0.9860,0.9622,0.9828,0.9724,0.9639,0.9640
1,0.9960,0.9938,0.9874,0.9958,0.9916,0.9889,0.9889
2,0.9890,0.9949,0.9706,0.9830,0.9767,0.9695,0.9696
3,0.9860,0.9918,0.9496,0.9912,0.9700,0.9608,0.9612
4,0.9850,0.9917,0.9620,0.9744,0.9682,0.9583,0.9583
5,0.9940,0.9916,0.9831,0.9915,0.9873,0.9833,0.9834
6,0.9860,0.9943,0.9538,0.9870,0.9701,0.9609,0.9612
7,0.9809,0.9882,0.9370,0.9824,0.9591,0.9467,0.9472
8,0.9860,0.9914,0.9580,0.9828,0.9702,0.9610,0.9612
9,0.9819,0.9865,0.9454,0.9783,0.9615,0.9497,0.9500


In [24]:
clf= finalize_model(lgb_tuned)

In [25]:
evaluate_model(clf)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [26]:
predictions= predict_model(clf, data=df_cache)
predictions

,niveau_satisfaction,derniere_evaluation,nb_projet,heures_par_mois,anciennete,accident_travail,promotion_5ans,departement,salaire,quitte,Label,Score
0,0.36,0.48,5,310,3,0,0,product_mng,normal,1,1,0.8946
1,0.41,0.48,2,136,3,0,0,technique,normal,1,1,0.9986
2,0.76,0.89,5,262,5,0,0,ventes,bas,1,1,0.9970
3,0.63,0.94,4,219,3,0,0,product_mng,bas,0,0,0.0010
4,0.98,0.86,4,254,2,1,0,R&D,bas,0,0,0.0001
...,...,...,...,...,...,...,...,...,...,...,...,...
745,0.15,0.53,4,205,5,1,0,ventes,bas,0,0,0.0009
746,0.10,0.94,7,281,4,0,0,technique,normal,1,1,1.0000
747,0.90,0.48,4,136,7,0,0,IT,bas,0,0,0.0016
748,0.72,0.78,3,229,2,0,0,ventes,normal,0,0,0.0015


In [27]:
save_model(clf, 'churn_fr_lgbm')

Transformation Pipeline and Model Succesfully Saved
